# 🏁 JRA Data Preprocessing (Tahap 1)

**Deskripsi:**
Notebook ini adalah gerbang utama dalam pipeline proyek Machine Learning UMA. Fokus utamanya adalah mengubah data mentah (*raw data*) balap kuda Jepang yang berantakan dan berbahasa Jepang menjadi format terstruktur yang bersih dan siap dianalisis.

**Tujuan Notebook Ini:**
1.  **Data Loading:** Memuat dataset mentah (`race_result.csv`).
2.  **Initial EDA:** Memeriksa kesehatan data (Missing values, duplikasi, tipe data).
3.  **Translation & Mapping:** Menerjemahkan kolom kategorikal penting (Cuaca, Jenis Kelamin, Kelas Balapan) dari Jepang ke Inggris menggunakan kamus data (*Dictionary*).
4.  **Data Cleaning (KDD):** * Konversi tipe data (`Date` ke Datetime).
    * Pembersihan kolom Target (`Rank`).
    * Penghapusan data *noise* (Kuda yang gagal finish).
5.  **Export:** Menyimpan hasil bersih ke `data/processed/race_result_cleaned.csv`.

## 1. Setup & Load Data
Tahap ini mengimpor pustaka penting (`pandas`, `numpy`) dan memuat file CSV mentah dari folder `data/raw`. Kita juga melakukan pengecekan file secara otomatis menggunakan `glob`.

In [1]:
import pandas as pd
import numpy as np
import os
import glob

# Konfigurasi Tampilan agar kolom tidak terpotong
pd.set_option('display.max_columns', 100)

# Cek file apa saja yang ada di folder raw
files = glob.glob('../data/raw/*.csv')
print("File ditemukan:", files)

File ditemukan: ['../data/raw\\19860105-20210731_laptime.csv', '../data/raw\\19860105-20210731_odds.csv', '../data/raw\\19860105-20210731_race_result.csv', '../data/raw\\20020615-20210731_corner_passing_order.csv']


### 📂 Smart Loading & Encoding Handling

Kode di bawah ini memiliki dua fungsi krusial:
1.  **Otomatisasi Seleksi File:** Mencari file yang mengandung kata `'race_result'` secara otomatis, sehingga kita tidak perlu mengetik nama file yang panjang secara manual.
2.  **Penanganan Karakter Jepang (Encoding):**
    Data teks dari Jepang seringkali tidak menggunakan standar internasional (`UTF-8`), melainkan standar lokal kuno seperti `Shift-JIS` atau `CP932` (Windows Japan).
    
    Kita menggunakan blok `try-except` bertingkat untuk mencoba membaca file dengan urutan prioritas:
    * **Prioritas 1:** `utf-8` (Standar Modern/Kaggle).
    * **Prioritas 2:** `shift_jis` (Standar Industri Jepang Lama).
    * **Prioritas 3:** `cp932` (Varian Shift-JIS khusus Microsoft Windows).

Ini menjamin data terbaca tanpa error *mojibake* (karakter aneh/rusak).

In [2]:
# Ambil file yang namanya mengandung 'race_result' atau file csv pertama jika tidak ada
target_file = [f for f in files if 'race_result' in f]
if target_file:
    file_path = target_file[0]
else:
    file_path = files[0] # Fallback ambil file pertama

print(f"Sedang membaca file: {file_path}")

try:
    # Coba baca dengan UTF-8 dulu (Standar Kaggle modern)
    df = pd.read_csv(file_path, encoding='utf-8')
    print("Sukses load dengan UTF-8!")
except:
    try:
        # Jika gagal, coba Shift-JIS (Standar Jepang lama)
        df = pd.read_csv(file_path, encoding='shift_jis')
        print("Sukses load dengan Shift-JIS!")
    except:
        # Terakhir coba cp932 (Varian Shift-JIS Windows)
        df = pd.read_csv(file_path, encoding='cp932')
        print("Sukses load dengan CP932!")

# Tampilkan 3 baris pertama
df.head(3)

Sedang membaca file: ../data/raw\19860105-20210731_race_result.csv


C:\Users\HP\AppData\Local\Temp\ipykernel_20360\4184097905.py:12: DtypeWarning: Columns (8,11,15,16,17,18,19,20,21,22,23,24,25,26,27,34,36,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='utf-8')


Sukses load dengan UTF-8!


,レース馬番ID,レースID,レース日付,開催回数,競馬場コード,競馬場名,開催日数,競争条件,レース記号/[抽],レース記号/(馬齢),レース記号/牝,レース記号/(父),レース記号/(別定),レース記号/(混),レース記号/(ハンデ),レース記号/(抽),レース記号/(市),レース記号/(定量),レース記号/牡,レース記号/関東配布馬,レース記号/(指),レース記号/関西配布馬,レース記号/九州産馬,レース記号/見習騎手,レース記号/せん,レース記号/(国際),レース記号/[指],レース記号/(特指),レース番号,重賞回次,レース名,リステッド・重賞競走,障害区分,芝・ダート区分,芝・ダート区分2,右左回り・直線区分,内・外・襷区分,距離(m),天候,馬場状態1,馬場状態2,発走時刻,着順,着順注記,枠番,馬番,馬名,性別,馬齢,斤量,騎手,タイム,着差,1コーナー,2コーナー,3コーナー,4コーナー,上り,単勝,人気,馬体重,場体重増減,東西・外国・地方区分,調教師,馬主,賞金(万円)
0,19860101010102,198601010101,1986-06-07,1,1,札幌,1,4歳以上300万下,[抽],(馬齢),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,4歳以上300万下,NaN,NaN,ダート,NaN,右,NaN,1500,晴,良,NaN,11:05,1.0,NaN,2,2,ワクセイ,牡,4,55.0,柏崎正次,1:34.3,NaN,NaN,7.0,3.0,1.0,38.6,2.1,1.0,468.0,0.0,東,宮沢今朝,アイ・ケイ・テイ・オーナーズ,290.0
1,19860101010103,198601010101,1986-06-07,1,1,札幌,1,4歳以上300万下,[抽],(馬齢),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,4歳以上300万下,NaN,NaN,ダート,NaN,右,NaN,1500,晴,良,NaN,11:05,2.0,NaN,3,3,マツタカラオー,牡,4,55.0,国兼正浩,1:36.0,大,NaN,4.0,3.0,4.0,40.3,7.0,4.0,430.0,4.0,東,斎藤籌敬,松井健一,120.0
2,19860101010105,198601010101,1986-06-07,1,1,札幌,1,4歳以上300万下,[抽],(馬齢),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,4歳以上300万下,NaN,NaN,ダート,NaN,右,NaN,1500,晴,良,NaN,11:05,3.0,NaN,5,5,カンキョウヘルス,牡,4,55.0,古小路重,1:36.2,1.1/4,NaN,1.0,2.0,3.0,40.6,59.1,6.0,460.0,-4.0,西,境直行,塚原金治,73.0


## 2. Initial Exploratory Data Analysis (EDA)
Sebelum memproses data, kita melakukan "Health Check" untuk memahami karakteristik data mentah:
* **Dimensi:** Berapa juta baris data yang kita punya?
* **Tipe Data:** Apakah angka terbaca sebagai teks?
* **Missing Values:** Seberapa parah data yang kosong?
* **Duplikasi:** Apakah ada data ganda yang perlu dibuang?

In [3]:
# 1. Cek Dimensi Data (Baris x Kolom)
print(f"\n[1] Dimensi Data: {df.shape[0]} baris, {df.shape[1]} kolom")


[1] Dimensi Data: 1626811 baris, 66 kolom


In [4]:
# 2. Cek Tipe Data & Penggunaan Memori
print("\n[2] Info Tipe Data:")
df.info()


[2] Info Tipe Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1626811 entries, 0 to 1626810
Data columns (total 66 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   レース馬番ID      1626811 non-null  int64  
 1   レースID        1626811 non-null  int64  
 2   レース日付        1626811 non-null  object 
 3   開催回数         1626811 non-null  int64  
 4   競馬場コード       1626811 non-null  int64  
 5   競馬場名         1626811 non-null  object 
 6   開催日数         1626811 non-null  int64  
 7   競争条件         1626811 non-null  object 
 8   レース記号/[抽]    12834 non-null    object 
 9   レース記号/(馬齢)   827790 non-null   object 
 10  レース記号/牝      208613 non-null   object 
 11  レース記号/(父)    23423 non-null    object 
 12  レース記号/(別定)   361406 non-null   object 
 13  レース記号/(混)    753618 non-null   object 
 14  レース記号/(ハンデ)  95331 non-null    object 
 15  レース記号/(抽)    6016 non-null     object 
 16  レース記号/(市)    6180 non-null     object 
 17  レース記号/(定量)   324044 non-n

In [5]:
# 3. Cek Missing Values (Data Kosong)
print("\n[3] Jumlah Missing Values per Kolom:")
missing_vals = df.isnull().sum()
print(missing_vals[missing_vals > 0])


[3] Jumlah Missing Values per Kolom:
レース記号/[抽]      1613977
レース記号/(馬齢)      799021
レース記号/牝        1418198
レース記号/(父)      1603388
レース記号/(別定)     1265405
レース記号/(混)       873193
レース記号/(ハンデ)    1531480
レース記号/(抽)      1620795
レース記号/(市)      1620631
レース記号/(定量)     1302767
レース記号/牡        1624410
レース記号/関東配布馬    1626023
レース記号/(指)      1596403
レース記号/関西配布馬    1625624
レース記号/九州産馬     1624349
レース記号/見習騎手     1619273
レース記号/せん       1626787
レース記号/(国際)     1584437
レース記号/[指]       758933
レース記号/(特指)     1479642
重賞回次           1562869
リステッド・重賞競走     1560707
障害区分           1572791
芝・ダート区分2       1593502
右左回り・直線区分        52336
内・外・襷区分        1451916
馬場状態2          1593502
着順               14915
着順注記           1611334
タイム              14584
着差              136760
1コーナー           929579
2コーナー           837495
3コーナー            18769
4コーナー             9376
上り               25754
単勝                6438
人気                6438
馬体重               4250
場体重増減             4250
賞金(万円)         1016845
dtype: int64


In [6]:
# 4. Cek Duplikasi Data
duplicate_count = df.duplicated().sum()
print(f"\n[4] Jumlah Baris Duplikat: {duplicate_count}")


[4] Jumlah Baris Duplikat: 0


In [7]:
# 5. Cek Integritas Kolom Kunci (Primary Keys)
if 'レースID' in df.columns and '馬番' in df.columns:
    unique_check = df.duplicated(subset=['レースID', '馬番']).sum()
    print(f"\n[5] Cek Konsistensi ID (Race ID + Horse No):")
    if unique_check == 0:
        print("    OK! Tidak ada duplikasi pada kombinasi Race ID dan Nomor Kuda.")
    else:
        print(f"    WARNING! Ada {unique_check} data ganda untuk kuda yang sama di race yang sama.")


[5] Cek Konsistensi ID (Race ID + Horse No):
    OK! Tidak ada duplikasi pada kombinasi Race ID dan Nomor Kuda.


## 3. Data Mapping & Translation (Jepang -> Inggris)
Data mentah JRA menggunakan karakter Kanji/Katakana. Agar model Machine Learning (dan kita) bisa memahaminya, kita melakukan *Mapping* manual.

**Proses yang dilakukan:**
1.  **Definisi Kamus:** Membuat Dictionary python untuk `sex` (Jantan/Betina), `surface` (Rumput/Tanah), `weather` (Cerah/Hujan), dll.
2.  **Renaming Columns:** Mengubah nama kolom Jepang (misal: `着順`) menjadi Inggris (`rank`).
3.  **Transformation:** Mengaplikasikan mapping tersebut ke DataFrame.

**Kelompok Fitur yang Diambil:**
* **Metadata:** Identitas balapan (ID, Tanggal, Tempat).
* **Track Info:** Kondisi lapangan (Rumput/Tanah, Cuaca, Jarak).
* **Profile:** Data fisik kuda, Joki, dan Pelatih.
* **Result (Target):** Posisi finis (`rank`) dan waktu lari.
* **Tactics:** Posisi tikungan (`corner_1` s.d `corner_4`) untuk analisis gaya lari (*Running Style*).

In [8]:
import pandas as pd
import numpy as np

# 1. Dictionary Mapping (KHUSUS Dataset Anda)
# Kiri: Header Jepang (dari tabel Anda) -> Kanan: Header Inggris Baru
column_mapping = {
    # --- Identitas Balapan (Race Metadata) ---
    'レースID': 'race_id',              # ID Unik
    'レース日付': 'date',                 # Tanggal (Penting untuk Time Series split)
    '発走時刻': 'start_time',           # Jam Lari (Pagi/Sore pengaruh ke suhu/track)
    '競馬場名': 'venue',                # Lokasi (Sapporo, Tokyo, dll)
    '開催回数': 'holding_number',       # Kai (Event ke-berapa musim ini)
    '開催日数': 'holding_days',         # Nichi (Hari ke-berapa dalam event)
    'レース名': 'race_name',            # Nama Lomba
    '競争条件': 'race_class',           # Kelas (Maiden, 1 Win Class, Open, dll) - PENTING
    'リステッド・重賞競走': 'race_type', # Tipe (Listed/Graded/Null)
    '重賞回次': 'grade_race_count',     # Hitungan ke-berapa untuk G1/G2
    
    # --- Kondisi Lintasan (Track Info) ---
    '芝・ダート区分': 'surface',          # Turf/Dirt
    '距離(m)': 'distance',              # Jarak
    '右左回り・直線区分': 'course_direction', # Kanan/Kiri/Lurus (Penting: kuda punya preferensi)
    '内・外・襷区分': 'course_track',     # Jalur Dalam/Luar (Penting untuk strategi)
    '天候': 'weather',                  # Cuaca
    '馬場状態1': 'track_condition',     # Kondisi Trek (Good/Heavy)
    '馬場状態2': 'track_condition_detail', # Detail (jarang ada, tapi ambil saja)

    # --- Profil Peserta (Horse/Jockey Profile) ---
    '馬名': 'horse_name',               # Nama Kuda
    '性別': 'sex',                      # Gender
    '馬齢': 'age',                      # Umur
    '東西・外国・地方区分': 'region',     # Asal (Timur/Barat/Asing) - PENTING (Kuda Kansai vs Kanto)
    '調教師': 'trainer',                # Pelatih
    '馬主': 'owner',                    # Pemilik (Sumber daya/uang pemilik pengaruh ke performa)
    '騎手': 'jockey',                   # Joki
    '斤量': 'jockey_weight',            # Beban bawaan
    '枠番': 'frame_no',                 # Nomor Frame (Posisi start pengaruh besar)
    '馬番': 'horse_no',                 # Nomor Punggung

    # --- Hasil & Taktik (Result & Tactics) ---
    '着順': 'rank',                     # Target Prediksi
    '着順注記': 'rank_note',            # Catatan (Diskualifikasi/Batal)
    'タイム': 'time',                   # Waktu Finish
    '着差': 'margin',                   # Jarak Kemenangan
    '上り': 'last_3f',                  # Kecepatan 600m terakhir (Stamina)
    
    # --- Taktik (Posisi Tikungan) ---
    # Ini fitur "Emas" untuk menentukan gaya lari (Nige/Senkou/Sashi/Oikomi)
    '1コーナー': 'corner_1',
    '2コーナー': 'corner_2',
    '3コーナー': 'corner_3',
    '4コーナー': 'corner_4',

    # --- Indikator Pasar (Betting) ---
    '単勝': 'odds',                     # Win Odds
    '人気': 'popularity',               # Favorit Pasar

    # --- Kondisi Fisik Harian ---
    '馬体重': 'horse_weight',           # Berat Badan
    '場体重増減': 'weight_change',        # Naik/Turun Berat (Kesehatan)
    '賞金(万円)': 'prize'                 # Hadiah (Indikator kelas kuda)
}

# 2. Rename Kolom dari Bahasa Jepang ke Inggris
df_clean = df.rename(columns=column_mapping)

# 3. FILTER PINTAR: Ambil hanya kolom yang ada di daftar mapping
target_cols = list(column_mapping.values())
df_clean = df_clean[target_cols]

# 4. Cek Hasil
print(f"Dimensi Data Bersih: {df_clean.shape}")
print("Header kolom sekarang:")
print(df_clean.columns.tolist())

# Tampilkan 3 baris teratas untuk verifikasi
df_clean.head(3)

Dimensi Data Bersih: (1626811, 41)
Header kolom sekarang:
['race_id', 'date', 'start_time', 'venue', 'holding_number', 'holding_days', 'race_name', 'race_class', 'race_type', 'grade_race_count', 'surface', 'distance', 'course_direction', 'course_track', 'weather', 'track_condition', 'track_condition_detail', 'horse_name', 'sex', 'age', 'region', 'trainer', 'owner', 'jockey', 'jockey_weight', 'frame_no', 'horse_no', 'rank', 'rank_note', 'time', 'margin', 'last_3f', 'corner_1', 'corner_2', 'corner_3', 'corner_4', 'odds', 'popularity', 'horse_weight', 'weight_change', 'prize']


,race_id,date,start_time,venue,holding_number,holding_days,race_name,race_class,race_type,grade_race_count,surface,distance,course_direction,course_track,weather,track_condition,track_condition_detail,horse_name,sex,age,region,trainer,owner,jockey,jockey_weight,frame_no,horse_no,rank,rank_note,time,margin,last_3f,corner_1,corner_2,corner_3,corner_4,odds,popularity,horse_weight,weight_change,prize
0,198601010101,1986-06-07,11:05,札幌,1,1,4歳以上300万下,4歳以上300万下,NaN,NaN,ダート,1500,右,NaN,晴,良,NaN,ワクセイ,牡,4,東,宮沢今朝,アイ・ケイ・テイ・オーナーズ,柏崎正次,55.0,2,2,1.0,NaN,1:34.3,NaN,38.6,NaN,7.0,3.0,1.0,2.1,1.0,468.0,0.0,290.0
1,198601010101,1986-06-07,11:05,札幌,1,1,4歳以上300万下,4歳以上300万下,NaN,NaN,ダート,1500,右,NaN,晴,良,NaN,マツタカラオー,牡,4,東,斎藤籌敬,松井健一,国兼正浩,55.0,3,3,2.0,NaN,1:36.0,大,40.3,NaN,4.0,3.0,4.0,7.0,4.0,430.0,4.0,120.0
2,198601010101,1986-06-07,11:05,札幌,1,1,4歳以上300万下,4歳以上300万下,NaN,NaN,ダート,1500,右,NaN,晴,良,NaN,カンキョウヘルス,牡,4,西,境直行,塚原金治,古小路重,55.0,5,5,3.0,NaN,1:36.2,1.1/4,40.6,NaN,1.0,2.0,3.0,59.1,6.0,460.0,-4.0,73.0


### Validasi Data Kategorikal (Pre-Mapping Check)

Sebelum kita menerjemahkan data dari Bahasa Jepang ke Inggris (atau mengonversinya menjadi angka), kita wajib melakukan **inspeksi isi data asli**.

**Mengapa langkah ini penting?**
1.  **Pembersihan Spasi (Trimming):** Data teks seringkali membawa "spasi hantu" di awal/akhir (contoh: `" Tokyo "` vs `"Tokyo"`). Kode `str.strip()` akan membersihkan ini agar mapping tidak error.
2.  **Audit Variasi Data:** Kita perlu tahu persis karakter apa yang dipakai.
    * *Contoh:* Apakah cuaca Cerah ditulis sebagai `晴` (1 huruf) atau `晴れ` (2 huruf)?
    * Apakah ada kode aneh (seperti `99` atau `Unknown`) yang perlu ditangani?

Hasil print dari kode ini akan menjadi **panduan utama** kita dalam membuat kamus terjemahan (*Dictionary Mapping*) di langkah selanjutnya.

In [9]:
# --- BLOCK: VALIDASI DAN MAPPING KATEGORIKAL ---

# Daftar kolom kategorikal yang akan kita proses
target_cat_cols = ['sex', 'weather', 'surface', 'track_condition', 'course_direction', 'region', 'venue', 'course_track', 'track_condition_detail', 'rank_note', 'race_class']

# ---------------------------------------------------------
# 1. CEK UNIQUE VALUES SEBELUM MAPPING (Data Asli)
# ---------------------------------------------------------

# Menghapus spasi kosong di awal/akhir teks
obj_cols = df_clean.select_dtypes(include=['object']).columns
for col in obj_cols:
    df_clean[col] = df_clean[col].str.strip()

print("=== [1] NILAI UNIK SEBELUM MAPPING (BAHASA JEPANG) ===")
for col in target_cat_cols:
    if col in df_clean.columns:
        unique_vals = df_clean[col].unique()
        print(f" > Kolom '{col}' ({len(unique_vals)} variasi): {unique_vals}")
    else:
        print(f" > Kolom '{col}' tidak ditemukan di DataFrame.")

=== [1] NILAI UNIK SEBELUM MAPPING (BAHASA JEPANG) ===
 > Kolom 'sex' (3 variasi): ['牡' '牝' 'セ']
 > Kolom 'weather' (6 variasi): ['晴' '曇' '雨' '小雨' '小雪' '雪']
 > Kolom 'surface' (2 variasi): ['ダート' '芝']
 > Kolom 'track_condition' (4 variasi): ['良' '稍重' '不良' '重']
 > Kolom 'course_direction' (4 variasi): ['右' nan '左' '直線']
 > Kolom 'region' (4 variasi): ['東' '西' '外' '地']
 > Kolom 'venue' (10 variasi): ['札幌' '函館' '福島' '新潟' '東京' '中山' '中京' '京都' '阪神' '小倉']
 > Kolom 'course_track' (7 variasi): [nan '外' '外-内' '襷' '内2周' '内-外' '外2周']
 > Kolom 'track_condition_detail' (5 variasi): [nan '良' '重' '不良' '稍重']
 > Kolom 'rank_note' (7 variasi): [nan '取' '中' '除' '失' '再' '降']
 > Kolom 'race_class' (49 variasi): ['4歳以上300万下' '3歳新馬' '4歳未勝利' '4歳以上400万下' '4歳以上900万下' '4歳以上オープン' '4歳以上600万下'
 '3歳未勝利' '3歳オープン' '障害4歳以上未勝利' '4歳900万下' '4歳400万下' '4歳以上1400万下'
 '障害4歳以上オープン' '4歳オープン' '障害4歳以上400万下' '3歳300万下' '3歳400万下' '4歳未出走' '4歳新馬'
 '4歳以上700万下' '4歳以上500万下' '4歳以上1500万下' '4歳500万下' '3歳500万下' '4歳以上800万下'
 '4歳以上1600万下' '5歳以上オー

In [10]:
# 2. DEFINISI KAMUS (DICTIONARY)
sex_map = {
    '牡': 'Male',
    '牝': 'Female',
    'セ': 'Gelding'
}

weather_map = {
    '晴': 'Sunny',
    '曇': 'Cloudy',
    '雨': 'Rain',
    '小雨': 'Light Rain',
    '雪': 'Snow',
    '小雪': 'Light Snow'
}

surface_map = {
    '芝': 'Turf',
    'ダート': 'Dirt',
    '障害': 'Steeplechase'
}

track_condition_map = {
    '良': 'Firm',      
    '稍重': 'Good',
    '重': 'Soft',
    '不良': 'Heavy'    
}

course_direction_map = {
    '右': 'Right',
    '左': 'Left',
    '直線': 'Straight'
}

region_map = {
    '東': 'East (Miho)',
    '西': 'West (Ritto)',
    '地': 'Local',
    '外': 'Foreign'
}

venue_map = {
    '札幌': 'Sapporo',
    '函館': 'Hakodate',
    '福島': 'Fukushima',
    '新潟': 'Niigata',
    '東京': 'Tokyo',
    '中山': 'Nakayama',
    '中京': 'Chukyo',
    '京都': 'Kyoto',
    '阪神': 'Hanshin',
    '小倉': 'Kokura'
}

course_track_map = {
    '外': 'Outer',
    '外-内': 'Outer to Inner',
    '襷': 'Jump Course (Tasuki)', 
    '内2周': 'Inner (2 Laps)',
    '内-外': 'Inner to Outer',
    '外2周': 'Outer (2 Laps)'
    # Note: NaN biasanya berarti 'Inner' (Jalur Dalam Standard)
}

rank_note_map = {
    '取': 'Scratched (Torikeshi)',   # Batal lari sebelum start
    '中': 'Aborted (Chushi)',        # Berhenti di tengah lomba (cedera dll)
    '除': 'Excluded (Jogai)',        # Dikeluarkan oleh panitia
    '失': 'Disqualified (Shikkaku)', # Diskualifikasi (pelanggaran berat)
    '再': 'Re-run',                  # Balapan ulang (sangat jarang)
    '降': 'Demoted (Kochaku)'        # Turun peringkat (pelanggaran ringan)
}

race_class_map = {
    # --- Newcomer / Maiden ---
    '2歳新馬': '2yo Newcomer',
    '3歳新馬': '3yo Newcomer',
    '4歳新馬': '4yo Newcomer',
    '2歳未勝利': '2yo Maiden',
    '3歳未勝利': '3yo Maiden',
    '4歳未勝利': '4yo Maiden',
    '3歳未出走': '3yo Unraced',
    '4歳未出走': '4yo Unraced',

    # --- 1 Win Class (Dulu 500 Juta Yen ke bawah) ---
    '2歳500万下': '2yo Under 5M',
    '3歳500万下': '3yo Under 5M',
    '4歳500万下': '4yo Under 5M',
    '3歳以上500万下': '3yo+ Under 5M',
    '4歳以上500万下': '4yo+ Under 5M',
    '2歳1勝クラス': '2yo 1 Win Class',
    '3歳1勝クラス': '3yo 1 Win Class',
    '3歳以上1勝クラス': '3yo+ 1 Win Class',
    '4歳以上1勝クラス': '4yo+ 1 Win Class',

    # --- 2 Win Class (Dulu 1000 Juta Yen ke bawah) ---
    '3歳900万下': '3yo Under 9M', # Kategori lama
    '4歳900万下': '4yo Under 9M',
    '3歳1000万下': '3yo Under 10M',
    '4歳以上900万下': '4yo+ Under 9M',
    '4歳以上1000万下': '4yo+ Under 10M',
    '3歳以上1000万下': '3yo+ Under 10M',
    '3歳以上2勝クラス': '3yo+ 2 Win Class',
    '4歳以上2勝クラス': '4yo+ 2 Win Class',

    # --- 3 Win Class (Dulu 1600 Juta Yen ke bawah) ---
    '4歳以上1400万下': '4yo+ Under 14M', # Kategori lama
    '4歳以上1500万下': '4yo+ Under 15M', # Kategori lama
    '3歳以上1600万下': '3yo+ Under 16M',
    '4歳以上1600万下': '4yo+ Under 16M',
    '4歳以上3勝クラス': '4yo+ 3 Win Class',

    # --- Open Class ---
    '2歳オープン': '2yo Open',
    '3歳オープン': '3yo Open',
    '4歳オープン': '4yo Open',
    '3歳以上オープン': '3yo+ Open',
    '4歳以上オープン': '4yo+ Open',
    '5歳以上オープン': '5yo+ Open',

    # --- Jump / Steeplechase (障害) ---
    '障害3歳以上未勝利': 'Jump 3yo+ Maiden',
    '障害4歳以上未勝利': 'Jump 4yo+ Maiden',
    '障害3歳以上オープン': 'Jump 3yo+ Open',
    '障害4歳以上オープン': 'Jump 4yo+ Open',
    '障害4歳以上400万下': 'Jump 4yo+ Under 4M',

    # --- Kelas Lama Lainnya (Jarang) ---
    '3歳300万下': '3yo Under 3M',
    '3歳400万下': '3yo Under 4M',
    '4歳400万下': '4yo Under 4M',
    '4歳以上300万下': '4yo+ Under 3M',
    '4歳以上400万下': '4yo+ Under 4M',
    '4歳以上600万下': '4yo+ Under 6M',
    '4歳以上700万下': '4yo+ Under 7M',
    '4歳以上800万下': '4yo+ Under 8M',
}

mappings = {
    'sex': sex_map,
    'weather': weather_map,
    'surface': surface_map,
    'track_condition': track_condition_map,
    'course_direction': course_direction_map,
    'region': region_map,
    'venue': venue_map,
    'course_track': course_track_map,
    'rank_note': rank_note_map,
    'race_class': race_class_map
}

In [11]:
# 3. EKSEKUSI MAPPING
for col, map_dict in mappings.items():
    if col in df_clean.columns:
        # Lakukan mapping
        # fillna(df_clean[col]) digunakan agar jika ada data yang TIDAK ada di kamus, 
        # dia tidak jadi NaN, tapi tetap huruf Jepang aslinya (agar bisa kita deteksi errornya)
        df_clean[col] = df_clean[col].map(map_dict).fillna(df_clean[col])

print(" > Mapping selesai.")

# 4. VERIFIKASI HASIL (CEK ULANG)
print("\n=== [HASIL AKHIR] NILAI UNIK SETELAH MAPPING ===")
target_cat_cols = ['sex', 'weather', 'surface', 'track_condition', 'course_direction', 'region', 'venue', 'course_track', 'track_condition_detail', 'rank_note', 'race_class']

for col in target_cat_cols:
    if col in df_clean.columns:
        vals = df_clean[col].unique()
        print(f"[{col}] -> {vals}")

 > Mapping selesai.

=== [HASIL AKHIR] NILAI UNIK SETELAH MAPPING ===
[sex] -> ['Male' 'Female' 'Gelding']
[weather] -> ['Sunny' 'Cloudy' 'Rain' 'Light Rain' 'Light Snow' 'Snow']
[surface] -> ['Dirt' 'Turf']
[track_condition] -> ['Firm' 'Good' 'Heavy' 'Soft']
[course_direction] -> ['Right' nan 'Left' 'Straight']
[region] -> ['East (Miho)' 'West (Ritto)' 'Foreign' 'Local']
[venue] -> ['Sapporo' 'Hakodate' 'Fukushima' 'Niigata' 'Tokyo' 'Nakayama' 'Chukyo'
 'Kyoto' 'Hanshin' 'Kokura']
[course_track] -> [nan 'Outer' 'Outer to Inner' 'Jump Course (Tasuki)' 'Inner (2 Laps)'
 'Inner to Outer' 'Outer (2 Laps)']
[track_condition_detail] -> [nan '良' '重' '不良' '稍重']
[rank_note] -> [nan 'Scratched (Torikeshi)' 'Aborted (Chushi)' 'Excluded (Jogai)'
 'Disqualified (Shikkaku)' 'Re-run' 'Demoted (Kochaku)']
[race_class] -> ['4yo+ Under 3M' '3yo Newcomer' '4yo Maiden' '4yo+ Under 4M'
 '4yo+ Under 9M' '4yo+ Open' '4yo+ Under 6M' '3yo Maiden' '3yo Open'
 'Jump 4yo+ Maiden' '4yo Under 9M' '4yo Under 4M' '4

In [12]:
df_clean.tail()

,race_id,date,start_time,venue,holding_number,holding_days,race_name,race_class,race_type,grade_race_count,surface,distance,course_direction,course_track,weather,track_condition,track_condition_detail,horse_name,sex,age,region,trainer,owner,jockey,jockey_weight,frame_no,horse_no,rank,rank_note,time,margin,last_3f,corner_1,corner_2,corner_3,corner_4,odds,popularity,horse_weight,weight_change,prize
1626806,202110030612,2021-07-18,16:15:00,Kokura,3,6,3歳以上1勝クラス,3yo+ 1 Win Class,NaN,NaN,Dirt,1700,Right,NaN,Cloudy,Firm,NaN,スズカキング,Male,4,West (Ritto),橋田満,永井啓弍,福永祐一,57.0,7,14,12.0,NaN,1:48.1,ハナ,39.2,15.0,16.0,16.0,15.0,17.0,7.0,470.0,14.0,NaN
1626807,202110030612,2021-07-18,16:15:00,Kokura,3,6,3歳以上1勝クラス,3yo+ 1 Win Class,NaN,NaN,Dirt,1700,Right,NaN,Cloudy,Firm,NaN,ヴォワドアンジェ,Female,3,West (Ritto),中竹和也,前田幸治,鮫島克駿,52.0,5,9,13.0,NaN,1:48.5,2.1/2,40.7,3.0,3.0,6.0,15.0,11.6,6.0,454.0,12.0,NaN
1626808,202110030612,2021-07-18,16:15:00,Kokura,3,6,3歳以上1勝クラス,3yo+ 1 Win Class,NaN,NaN,Dirt,1700,Right,NaN,Cloudy,Firm,NaN,マクシミリアン,Male,3,West (Ritto),奥村豊,ターフ・スポート,城戸義政,54.0,3,5,14.0,NaN,1:48.5,クビ,40.2,9.0,9.0,13.0,9.0,165.4,16.0,442.0,0.0,NaN
1626809,202110030612,2021-07-18,16:15:00,Kokura,3,6,3歳以上1勝クラス,3yo+ 1 Win Class,NaN,NaN,Dirt,1700,Right,NaN,Cloudy,Firm,NaN,アスクスタイルマン,Male,3,West (Ritto),矢作芳人,廣崎利洋ＨＤ,岩田望来,54.0,8,16,15.0,NaN,1:48.6,1/2,40.4,13.0,12.0,11.0,9.0,10.7,5.0,470.0,10.0,NaN
1626810,202110030612,2021-07-18,16:15:00,Kokura,3,6,3歳以上1勝クラス,3yo+ 1 Win Class,NaN,NaN,Dirt,1700,Right,NaN,Cloudy,Firm,NaN,キワミ,Gelding,4,West (Ritto),昆貢,寺田寿男,太宰啓介,57.0,6,12,16.0,NaN,1:48.9,1.1/2,41.2,3.0,4.0,6.0,12.0,69.7,14.0,470.0,-2.0,NaN


### High-Performance Translation

Pada tahap ini, kita melakukan pemrosesan data tingkat lanjut yang melibatkan manipulasi string berat.

**Tantangan Utama:**
Kita memiliki jutaan baris data dengan nama Kuda/Joki dalam Huruf Kanji. Menerjemahkannya satu per satu (row-by-row) akan memakan waktu berjam-jam.

**Solusi & Logika Kode:**
1.  **Fast Time Conversion:** Menggunakan logika *List Comprehension* (Python native) untuk mengubah format waktu `1:30.5` (menit:detik) menjadi detik total (`90.5`). Cara ini terbukti lebih cepat daripada fungsi `.apply()` bawaan Pandas untuk operasi string sederhana.
2.  **Rank Cleaning:** Memaksa kolom peringkat menjadi angka. Status seperti "Did Not Finish" atau "Disqualified" diubah menjadi `NaN`.
3.  **Optimized Translation (The Smart Way):**
    * Alih-alih menerjemahkan jutaan baris, kita hanya mengambil **Nilai Unik (Unique Values)** dari nama kuda/joki.
    * Contoh: Dari 1 juta data balapan, mungkin hanya ada 50.000 kuda unik.
    * Kita menerjemahkan 50.000 nama unik tersebut menggunakan library `pykakasi`, menyimpannya dalam kamus (*Dictionary*), lalu memetakan (*Map*) kembali ke data utama.
    * **Hasil:** Proses yang tadinya butuh 5 menit bisa selesai dalam 15 detik.

**Output Akhir:**
Dataframe `df_final` yang bersih, memiliki kolom nama dalam Bahasa Inggris (Romaji), dan siap untuk analisis.

In [13]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import pykakasi

# Inisialisasi tqdm
tqdm.pandas()

# --- 1. Konversi Waktu (Optimasi) ---
# Menggunakan logika Python murni dalam list comprehension seringkali lebih cepat 
# daripada .apply() pandas untuk operasi string sederhana pada jumlah data besar.
def fast_time_conversion(time_series):
    def convert(t):
        if pd.isna(t) or t == '': return None
        t = str(t)
        try:
            if ':' in t:
                m, s = t.split(':')
                return float(m) * 60 + float(s)
            return float(t)
        except:
            return None
    
    # List comprehension lebih cepat dibanding .apply
    return [convert(x) for x in time_series]

print("2. Konversi waktu...")
df_clean['time_seconds'] = fast_time_conversion(df_clean['time'])


# --- 2. Bersihkan Rank ---
df_clean['rank'] = pd.to_numeric(df_clean['rank'], errors='coerce')


# --- 3. Translasi Nama (OPTIMASI BESAR - Unique Mapping) ---
try:
    kks = pykakasi.kakasi()
    
    # Fungsi translasi dasar
    def jp_to_romaji_single(text):
        if not isinstance(text, str): return text
        result = kks.convert(text)
        return " ".join([item['hepburn'] for item in result]).title()

    # Fungsi wrapper untuk translasi masal yang efisien
    def translate_column_efficiently(df, col_name, new_col_name):
        print(f"   > Memproses kolom: {col_name}...")
        
        # A. Ambil nilai unik saja (buang duplikat & NaN)
        unique_names = df[col_name].dropna().unique()
        
        # B. Translasi nilai unik (Jauh lebih sedikit dibanding total baris)
        # Menggunakan dictionary comprehension
        name_map = {
            name: jp_to_romaji_single(name) 
            for name in tqdm(unique_names, desc=f"Translating unique {col_name}")
        }
        
        # C. Map kembali ke DataFrame utama
        df[new_col_name] = df[col_name].map(name_map)

    print("\n3. Menerjemahkan Nama (Metode Cepat)...")
    
    # Eksekusi untuk setiap kolom target
    target_cols = {
        'horse_name': 'horse_name_eng',
        'jockey': 'jockey_eng',
        'owner': 'owner_eng',
        'trainer':'trainer_eng',
        'race_name': 'race_name_eng'
    }
    
    for jp_col, eng_col in target_cols.items():
        if jp_col in df_clean.columns:
            translate_column_efficiently(df_clean, jp_col, eng_col)
        else:
            print(f"   [Skip] Kolom {jp_col} tidak ditemukan.")

except ImportError:
    print("Library pykakasi belum terinstall, nama tetap Kanji.")
except Exception as e:
    print(f"Terjadi error: {e}")


# --- 4. Finalisasi ---
print("\n--- Data Siap ---")
# Drop data kotor
df_final = df_clean.dropna(subset=['rank', 'time_seconds']).copy() # .copy() agar tidak warning settingwithcopy

print(f"Total Baris Bersih: {len(df_final)}")
display(df_final[['date', 'venue', 'horse_name_eng', 'sex', 'rank', 'time_seconds']].head())

2. Konversi waktu...


d:\KULIAH\FINAL PROJECT\Project-UMA\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



3. Menerjemahkan Nama (Metode Cepat)...
   > Memproses kolom: horse_name...


Translating unique horse_name: 100%|██████████| 147495/147495 [00:14<00:00, 10450.95it/s]


   > Memproses kolom: jockey...


...

Translating unique trainer: 100%|██████████| 1332/1332 [00:00<00:00, 5310.45it/s]


   > Memproses kolom: race_name...


Translating unique race_name: 100%|██████████| 2330/2330 [00:00<00:00, 6591.75it/s]



--- Data Siap ---
Total Baris Bersih: 1611896


,date,venue,horse_name_eng,sex,rank,time_seconds
0,1986-06-07,Sapporo,Wakusei,Male,1.0,94.3
1,1986-06-07,Sapporo,Matsutakaraoo,Male,2.0,96.0
2,1986-06-07,Sapporo,Kankyouherusu,Male,3.0,96.2
3,1986-06-07,Sapporo,Suzutakaeesu,Male,4.0,96.6
4,1986-06-07,Sapporo,Kuriyaapaaman,Male,5.0,97.4


In [14]:
df_final.head(3)

,race_id,date,start_time,venue,holding_number,holding_days,race_name,race_class,race_type,grade_race_count,surface,distance,course_direction,course_track,weather,track_condition,track_condition_detail,horse_name,sex,age,region,trainer,owner,jockey,jockey_weight,frame_no,horse_no,rank,rank_note,time,margin,last_3f,corner_1,corner_2,corner_3,corner_4,odds,popularity,horse_weight,weight_change,prize,time_seconds,horse_name_eng,jockey_eng,owner_eng,trainer_eng,race_name_eng
0,198601010101,1986-06-07,11:05,Sapporo,1,1,4歳以上300万下,4yo+ Under 3M,NaN,NaN,Dirt,1500,Right,NaN,Sunny,Firm,NaN,ワクセイ,Male,4,East (Miho),宮沢今朝,アイ・ケイ・テイ・オーナーズ,柏崎正次,55.0,2,2,1.0,NaN,1:34.3,NaN,38.6,NaN,7.0,3.0,1.0,2.1,1.0,468.0,0.0,290.0,94.3,Wakusei,Kashiwazaki Masaji,Ai・Kei・Tei・Oonaazu,Miyazawa Kesa,4 Toshi Ijou 300 Man Shita
1,198601010101,1986-06-07,11:05,Sapporo,1,1,4歳以上300万下,4yo+ Under 3M,NaN,NaN,Dirt,1500,Right,NaN,Sunny,Firm,NaN,マツタカラオー,Male,4,East (Miho),斎藤籌敬,松井健一,国兼正浩,55.0,3,3,2.0,NaN,1:36.0,大,40.3,NaN,4.0,3.0,4.0,7.0,4.0,430.0,4.0,120.0,96.0,Matsutakaraoo,Kuni Ken Masahiro,Matsui Ken'Ichi,Saitou Chuu Takashi,4 Toshi Ijou 300 Man Shita
2,198601010101,1986-06-07,11:05,Sapporo,1,1,4歳以上300万下,4yo+ Under 3M,NaN,NaN,Dirt,1500,Right,NaN,Sunny,Firm,NaN,カンキョウヘルス,Male,4,West (Ritto),境直行,塚原金治,古小路重,55.0,5,5,3.0,NaN,1:36.2,1.1/4,40.6,NaN,1.0,2.0,3.0,59.1,6.0,460.0,-4.0,73.0,96.2,Kankyouherusu,Ko Kouji Juu,Tsukahara Kinji,Sakai Chokkou,4 Toshi Ijou 300 Man Shita


## 4. Data Cleaning (KDD Standard)
Ini adalah inti dari proses pembersihan data (*Knowledge Discovery in Databases*).

**Langkah-langkah:**
1.  **Type Conversion:** Mengubah kolom `date` menjadi objek `datetime` agar kita bisa menghitung selisih hari nantinya.
2.  **Noise Removal (Target Variable):** * Kolom `rank` (Posisi Finis) di data mentah sering berisi teks seperti "Batal" atau "Gagal".
    * Kita memaksa kolom ini menjadi **Numerik**. Teks sampah otomatis diubah menjadi `NaN`.
3.  **Handling Missing Values:**
    * **Aturan Tegas:** Jika seekor kuda tidak memiliki `rank` (Posisi Finis) atau `time_seconds` (Waktu Lari), maka data tersebut **DIBUANG**.
    * *Alasan:* Kita tidak bisa melatih model Supervised Learning tanpa target label yang jelas.

In [15]:
print("--- Memulai Data Cleaning (KDD Standard) ---")

# 1. TYPE CONVERSION (Perbaikan Format Data)
# ------------------------------------------------
# Target: Mengubah 'date' dari string ke datetime objek agar bisa diurutkan/dihitung selisih harinya
df_clean['date'] = pd.to_datetime(df_clean['date'])
print(" [✓] Konversi tipe data 'date' ke datetime selesai.")


# 2. NOISE REMOVAL (Penghapusan Data Sampah)
# ------------------------------------------------
# Target: Kolom 'rank' kadang berisi teks Jepang ("Batal", "Gagal") yang tidak berguna.
# Kita paksa jadi angka. Jika teks, dia akan berubah jadi NaN (Not a Number).
df_clean['rank'] = pd.to_numeric(df_clean['rank'], errors='coerce')

# Pastikan 'time_seconds' juga numerik (sudah dilakukan di tahap sebelumnya, tapi kita double check)
# Data noise (seperti kuda yang tidak finish) pasti tidak punya waktu.
print(" [✓] Konversi 'rank' ke numerik selesai (Teks sampah jadi NaN).")


# 3. HANDLING MISSING VALUES (Penanganan Data Kosong)
# ------------------------------------------------
# Strategi: 
# - Untuk Target (Rank & Time): DELETE (Karena tidak bisa dipakai training).
# - Untuk Fitur Lain (Kuda/Joki): KEEP (Sudah di-handle via mapping/fillna sebelumnya).

before_drop = len(df_clean)
# Drop baris dimana 'rank' ATAU 'time_seconds' kosong (NaN)
df_final = df_clean.dropna(subset=['rank', 'time_seconds']).copy()
after_drop = len(df_final)

# Finishing Touch: Ubah rank jadi integer (1.0 -> 1) agar rapi
df_final['rank'] = df_final['rank'].astype(int)

print(f" [✓] Handling Missing Values selesai.")
print(f"     > Data Awal   : {before_drop} baris")
print(f"     > Data Dibuang: {before_drop - after_drop} baris (Noise/Gagal Finish)")
print(f"     > Data Bersih : {after_drop} baris")

# Cek Info Akhir
print("\n[Info Tipe Data Final]")
print(df_final[['date', 'rank', 'time_seconds']].dtypes)

--- Memulai Data Cleaning (KDD Standard) ---
 [✓] Konversi tipe data 'date' ke datetime selesai.
 [✓] Konversi 'rank' ke numerik selesai (Teks sampah jadi NaN).
 [✓] Handling Missing Values selesai.
     > Data Awal   : 1626811 baris
     > Data Dibuang: 14915 baris (Noise/Gagal Finish)
     > Data Bersih : 1611896 baris

[Info Tipe Data Final]
date            datetime64[ns]
rank                     int64
time_seconds           float64
dtype: object


## 5. Validasi Data & Export
Sebagai validasi terakhir, kita mencoba mencari riwayat balapan kuda legendaris (contoh: *Silence Suzuka*) untuk memastikan data sudah bersih dan akurat.

Setelah validasi sukses, data bersih disimpan ke folder `processed` agar siap digunakan oleh **Notebook 02 (Feature Engineering)**.

In [16]:
# Keyword pencarian 
keyword = "サイレンススズカ"

# Filter DataFrame
# na=False agar tidak error jika ada nama yang kosong
search_results = df_final[df_final['horse_name'].str.contains(keyword, case=False, na=False)]

# Urutkan berdasarkan tanggal balapan
search_results = search_results.sort_values('date')

# Tampilkan kolom-kolom penting
cols_to_show = ['date', 'race_name_eng','race_name_eng','race_class','owner_eng', 'horse_name_eng', 'jockey_eng', 'trainer_eng','rank', 'odds', 'popularity']

print(f"Ditemukan {len(search_results)} data balapan dengan nama '{keyword}':")
display(search_results[cols_to_show])

Ditemukan 14 data balapan dengan nama 'サイレンススズカ':


,date,race_name_eng,race_name_eng,race_class,owner_eng,horse_name_eng,jockey_eng,trainer_eng,rank,odds,popularity
464640,1997-02-01,4 Toshi Shinba,4 Toshi Shinba,4yo Newcomer,Nagai Kei Ni,Sairensusuzuka,Kamimura Youkou,Hashida Man,1,1.3,1.0
454595,1997-03-02,Houchi Hai Yayoishou,Houchi Hai Yayoishou,4yo Open,Nagai Kei Ni,Sairensusuzuka,Kamimura Youkou,Hashida Man,8,3.5,2.0
470971,1997-04-05,4 Toshi 500 Man Shita,4 Toshi 500 Man Shita,4yo Under 5M,Nagai Kei Ni,Sairensusuzuka,Kamimura Youkou,Hashida Man,1,1.2,1.0
448898,1997-05-10,Purinshiparu S,Purinshiparu S,4yo Open,Nagai Kei Ni,Sairensusuzuka,Kamimura Youkou,Hashida Man,1,2.3,2.0
450053,1997-06-01,Toukyouyuushun,Toukyouyuushun,4yo Open,Nagai Kei Ni,Sairensusuzuka,Kamimura Youkou,Hashida Man,9,8.6,4.0
473348,1997-09-14,Koubeshinbunhai,Koubeshinbunhai,4yo Open,Nagai Kei Ni,Sairensusuzuka,Kamimura Youkou,Hashida Man,2,2.1,1.0
451501,1997-10-26,Tennoushou ( Aki ),Tennoushou ( Aki ),4yo+ Open,Nagai Kei Ni,Sairensusuzuka,Kawauchi Hiroshi,Hashida Man,6,17.6,4.0
468906,1997-11-16,Mairuchanpion S,Mairuchanpion S,4yo+ Open,Nagai Kei Ni,Sairensusuzuka,Kawauchi Hiroshi,Hashida Man,15,19.1,6.0
491465,1998-02-14,Barentain S,Barentain S,4yo+ Open,Nagai Kei Ni,Sairensusuzuka,Taketoyo,Hashida Man,1,1.5,1.0
499193,1998-03-15,Nakayamakinen,Nakayamakinen,4yo+ Open,Nagai Kei Ni,Sairensusuzuka,Taketoyo,Hashida Man,1,1.4,1.0


In [17]:
# Simpan ke folder processed agar siap dipakai modeling
output_path = '../data/processed/race_result_cleaned.csv'
os.makedirs('../data/processed', exist_ok=True)

df_final.to_csv(output_path, index=False)
print(f"Data bersih tersimpan di: {output_path}")

Data bersih tersimpan di: ../data/processed/race_result_cleaned.csv
